In [1]:
#import libraries
import json
import psycopg2
import pandas as pd
import numpy as np

In [2]:
#Establish connection to database
with open('config.json') as f:
    conf = json.load(f)
conn_str = "host={} dbname={} user={} password={}".format(conf['host'], conf['database'], conf['user'], conf['passw'])
conn = psycopg2.connect(conn_str)

In [3]:
#Read columns from 2017 table
df = pd.read_sql('SELECT loan_amnt, term, int_rate, emp_length, home_ownership, annual_inc, purpose, fico_range_low, fico_range_high, application_type FROM loanstats_2017', con=conn)

In [5]:
#Total number of elements in the loaded dataFrame
print ("Original size is",df.size)
df.head(3)

Original size is 4435790


,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type
0,30000,36 months,7.35%,3 years,RENT,100000,debt_consolidation,715,719,Individual
1,6500,36 months,14.08%,< 1 year,RENT,30000,debt_consolidation,685,689,Individual
2,10000,36 months,10.91%,4 years,RENT,50000,debt_consolidation,695,699,Individual


In [7]:
#Check on datatypes in database
#Note : Object in Pandas is a  String
df.dtypes

loan_amnt           object
term                object
int_rate            object
emp_length          object
home_ownership      object
annual_inc          object
purpose             object
fico_range_low      object
fico_range_high     object
application_type    object
dtype: object

In [9]:
#Drop from the dataframe rows that have 'application_type' value different than "Individual"
df1 = df.drop(df[df['application_type'] != 'Individual'].index)
print ("The new size is",df1.size)

The new size is 4011540


In [42]:
#Sometimes the null values in this dataSet are not as NaN(pandas null value), but they are actually a string like 'n/a', 'NONE', 'null', 'NULL'
#Replacing them by NaN, so pandas can detect that we have a null value

df1.replace({'N/A':np.NaN,'n/a':np.NaN}, inplace=True)
df1.replace({'NONE':np.NaN,'none':np.NaN}, inplace=True)
df1.replace({'NULL':np.NaN,'null':np.NaN}, inplace=True)



In [43]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401154 entries, 0 to 443578
Data columns (total 10 columns):
loan_amnt           401154 non-null object
term                401154 non-null int64
int_rate            401154 non-null object
emp_length          401154 non-null int32
home_ownership      401153 non-null object
annual_inc          401154 non-null object
purpose             401154 non-null object
fico_range_low      401154 non-null object
fico_range_high     401154 non-null object
application_type    401154 non-null object
dtypes: int32(1), int64(1), object(8)
memory usage: 32.1+ MB


In [174]:
#Get List of unique values in the 'term' column
df1['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [17]:
#Replace 'term' column by valid values
df1['term'] = df1['term'].map({' 36 months':'36', ' 60 months':'60'})

In [18]:
#convert to numeric
df1['term'] = pd.to_numeric(df1['term'])

In [21]:
df1['term'].unique().shape

(2,)

In [20]:
#Check for null values
df1[df1['term'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [179]:
#Get list of unique values in 'int_rate' column
df1['int_rate'].unique()

array(['7.35%', '14.08%', '10.91%', '9.93%', '19.03%', ' 14.08%',
       ' 15.05%', '  7.21%', ' 13.59%', ' 10.42%', ' 12.62%', '18.06%',
       '7.97%', '9.44%', '  9.93%', '  9.44%', ' 26.30%', '13.59%',
       '12.62%', '  7.97%', '11.99%', '15.05%', '7.07%', '10.42%',
       '16.02%', '7.21%', '26.30%', '30.84%', ' 10.91%', '  7.35%',
       ' 30.79%', '17.09%', '5.32%', ' 22.91%', '23.88%', '  5.32%',
       ' 11.99%', ' 16.02%', ' 20.00%', ' 23.88%', ' 18.06%', '30.94%',
       '20.00%', ' 21.45%', ' 17.09%', ' 19.03%', '25.82%', '22.91%',
       '21.45%', '28.72%', '24.85%', '  7.07%', ' 25.82%', ' 29.69%',
       ' 30.65%', '29.69%', ' 24.85%', ' 30.94%', ' 28.72%', ' 30.99%',
       ' 30.84%', '30.75%', ' 30.89%', '30.65%', '30.89%', '30.79%',
       ' 30.75%', '30.99%', '30.17%', ' 30.17%', '6.08%', '6.72%',
       '6.00%', ' 14.99%', '  7.49%', ' 15.99%', ' 18.99%', ' 12.74%',
       '  7.24%', '  7.99%', ' 11.39%', ' 13.99%', ' 19.99%', '  8.24%',
       ' 13.49%', ' 10.49%

In [44]:
#Remove '%' character in 'int_rate' column
#df1['int_rate'] = df1['int_rate'].str.replace('%' , '')
df1['int_rate'].replace(to_replace='%', value='', inplace=True, regex=True)

In [45]:
#Remove spaces in 'int_rate' column
#df1['int_rate'] = df1['int_rate'].str.replace('\s+', '')
df1['int_rate'].replace(to_replace='\s+', value='', inplace=True, regex=True)

In [182]:
df1['int_rate'].unique()

array(['7.35', '14.08', '10.91', '9.93', '19.03', '15.05', '7.21',
       '13.59', '10.42', '12.62', '18.06', '7.97', '9.44', '26.30',
       '11.99', '7.07', '16.02', '30.84', '30.79', '17.09', '5.32',
       '22.91', '23.88', '20.00', '30.94', '21.45', '25.82', '28.72',
       '24.85', '29.69', '30.65', '30.99', '30.75', '30.89', '30.17',
       '6.08', '6.72', '6.00', '14.99', '7.49', '15.99', '18.99', '12.74',
       '7.24', '7.99', '11.39', '13.99', '19.99', '8.24', '13.49',
       '10.49', '11.44', '16.99', '11.49', '22.74', '25.49', '28.69',
       '17.99', '29.49', '26.24', '30.49', '6.99', '21.49', '23.99',
       '24.74', '29.99', '30.74'], dtype=object)

In [183]:
#Convert 'int_rate' data type to numeric
df1['int_rate'] = pd.to_numeric(df1['int_rate'])

In [184]:
df1['int_rate'].unique()

array([ 7.35, 14.08, 10.91,  9.93, 19.03, 15.05,  7.21, 13.59, 10.42,
       12.62, 18.06,  7.97,  9.44, 26.3 , 11.99,  7.07, 16.02, 30.84,
       30.79, 17.09,  5.32, 22.91, 23.88, 20.  , 30.94, 21.45, 25.82,
       28.72, 24.85, 29.69, 30.65, 30.99, 30.75, 30.89, 30.17,  6.08,
        6.72,  6.  , 14.99,  7.49, 15.99, 18.99, 12.74,  7.24,  7.99,
       11.39, 13.99, 19.99,  8.24, 13.49, 10.49, 11.44, 16.99, 11.49,
       22.74, 25.49, 28.69, 17.99, 29.49, 26.24, 30.49,  6.99, 21.49,
       23.99, 24.74, 29.99, 30.74])

In [185]:
#Check for null values
df1[df1['int_rate'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [186]:
df1['int_rate'].dtype

dtype('float64')

In [187]:
df1['loan_amnt'].unique()

array(['30000', '6500', '10000', ..., '37625', '38775', '36775'],
      dtype=object)

In [188]:
#convert 'loan_amnt' to numeric
df1['loan_amnt'] = pd.to_numeric(df1['loan_amnt'])

In [189]:
df1['loan_amnt'].dtype

dtype('int64')

In [190]:
df1['loan_amnt'].unique()

array([30000,  6500, 10000, ..., 37625, 38775, 36775], dtype=int64)

In [191]:
#Check for null values
df1[df1['loan_amnt'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [192]:
df1['annual_inc'].unique()

array(['100000', '30000', '50000', ..., '112740', '14112', '107368.9'],
      dtype=object)

In [193]:
#Convert annual_inc to numeric
df1['annual_inc'] = pd.to_numeric(df1['annual_inc'])

In [194]:
df1['annual_inc'].dtype

dtype('float64')

In [195]:
df1['annual_inc'].unique()

array([100000. ,  30000. ,  50000. , ..., 112740. ,  14112. , 107368.9])

In [196]:
#Check for null values
df1[df1['annual_inc'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [197]:
df1['fico_range_low'].unique()

array(['715', '685', '695', '660', '665', '725', '705', '755', '700',
       '710', '690', '745', '670', '720', '760', '750', '680', '675',
       '735', '730', '780', '770', '785', '825', '740', '765', '800',
       '790', '775', '810', '795', '830', '835', '805', '820', '815',
       '840', '845'], dtype=object)

In [198]:
#Convert fico low to numeric
df1['fico_range_low'] = pd.to_numeric(df1['fico_range_low'])

In [199]:
df1['fico_range_low'].dtype

dtype('int64')

In [200]:
df1['fico_range_low'].unique()

array([715, 685, 695, 660, 665, 725, 705, 755, 700, 710, 690, 745, 670,
       720, 760, 750, 680, 675, 735, 730, 780, 770, 785, 825, 740, 765,
       800, 790, 775, 810, 795, 830, 835, 805, 820, 815, 840, 845],
      dtype=int64)

In [201]:
#Check for null values
df1[df1['fico_range_low'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [202]:
#Convert fico high to numeric
df1['fico_range_high'] = pd.to_numeric(df1['fico_range_high'])

In [203]:
df1['fico_range_high'].unique()

array([719, 689, 699, 664, 669, 729, 709, 759, 704, 714, 694, 749, 674,
       724, 764, 754, 684, 679, 739, 734, 784, 774, 789, 829, 744, 769,
       804, 794, 779, 814, 799, 834, 839, 809, 824, 819, 844, 850],
      dtype=int64)

In [204]:
#Check for null values
df1[df1['fico_range_high'].isnull()]



,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [205]:
#I think we need to make the home_ownership and purpose columns numerical as well

In [22]:
#Check for null values in emp_length
df1[df1['emp_length'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type
16,28000,60,18.06%,NaN,MORTGAGE,93996,major_purchase,700,704,Individual
21,20000,36,7.21%,NaN,MORTGAGE,40000,debt_consolidation,710,714,Individual
54,4800,36,7.21%,NaN,RENT,17850,debt_consolidation,715,719,Individual
56,12000,36,11.99%,NaN,MORTGAGE,55555,debt_consolidation,710,714,Individual
62,15175,36,7.97%,NaN,MORTGAGE,73200,debt_consolidation,705,709,Individual
74,25000,36,7.97%,NaN,OWN,82000,debt_consolidation,735,739,Individual
110,5000,36,7.07%,NaN,OWN,45000,debt_consolidation,785,789,Individual
132,5500,36,17.09%,NaN,MORTGAGE,43200,home_improvement,700,704,Individual
141,3000,36,10.91%,NaN,RENT,19500,car,695,699,Individual
148,16000,36,11.99%,NaN,OWN,60000,debt_consolidation,745,749,Individual


In [30]:
df1['emp_length'].unique()

array(['3 years', '< 1 year', '4 years', '7 years', '10+ years',
       '2 years', '1 year', '5 years', nan, '9 years', '6 years',
       '8 years'], dtype=object)

In [37]:
#Convert emp_length to numerical
df1['emp_length'].fillna(value=0,inplace=True)
df1['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df1['emp_length'] = df1['emp_length'].astype(int)

In [38]:
df1['emp_length'].unique()

array([ 3,  1,  4,  7, 10,  2,  5,  0,  9,  6,  8], dtype=int64)

In [213]:
#check for null values
df1[df1['home_ownership'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type
352330,10000,36,12.62,NaN,NaN,92000.0,debt_consolidation,680,684,Individual


In [214]:
#We have one record in home_ownership that is null as you can see above, what do we do with it?

In [215]:
df1['home_ownership'].unique()

array(['RENT', 'MORTGAGE', 'OWN', 'ANY', nan], dtype=object)

In [216]:
df1[df1['purpose'].isnull()]

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,fico_range_low,fico_range_high,application_type


In [217]:
df1['purpose'].unique()

array(['debt_consolidation', 'other', 'home_improvement',
       'major_purchase', 'credit_card', 'small_business', 'car',
       'vacation', 'medical', 'moving', 'house', 'renewable_energy',
       'educational', 'wedding'], dtype=object)